<a href="https://colab.research.google.com/github/rodrigofer89/cybersecurity/blob/main/Checkpoint_2_CyberSecurity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyngrok
!pip install -q streamlit
!pip install -q streamlit_ace
!pip install SpeechRecognition
!pip install PyAudio

     |████████████████████████████████| 745 kB 14.3 MB/s 
     |████████████████████████████████| 9.2 MB 14.8 MB/s 
     |████████████████████████████████| 237 kB 74.3 MB/s 
     |████████████████████████████████| 4.7 MB 49.7 MB/s 
     |████████████████████████████████| 164 kB 70.8 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████████████████████████████| 182 kB 50.7 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 51 kB 7.2 MB/s 
     |████████████████████████████████| 3.6 MB 14.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metada

In [ ]:
!git clone https://github.com/stefanrmmr/streamlit_audio_recorder.git

Cloning into 'streamlit_audio_recorder'...
remote: Enumerating objects: 37457, done.
remote: Counting objects: 100% (883/883), done.
remote: Compressing objects: 100% (482/482), done.
remote: Total 37457 (delta 413), reused 558 (delta 383), pack-reused 36574
Receiving objects: 100% (37457/37457), 55.64 MiB | 20.10 MiB/s, done.
Resolving deltas: 100% (8636/8636), done.
Checking out files: 100% (37304/37304), done.


In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
%%writefile app.py

import streamlit as st
from pyngrok import ngrok
import numpy as np
import speech_recognition as sr
import soundfile as sf
import datetime
import streamlit.components.v1 as components
from io import BytesIO
import librosa
import re
import os

parent_dir = os.path.dirname(os.path.abspath(__file__))
build_dir = os.path.join(parent_dir, "streamlit_audio_recorder/st_audiorec/frontend/build")
st_audiorec = components.declare_component("st_audiorec", path=build_dir)

#Cria website
def main():

  #Elementos da webpage
  html_temp = """
  <div>
    <div style ="width: 34%; margin-right: 1%; height: 100px; display: inline-block; float: left; background-image: url(https://www.santander.com.br/sites/WPC_CMS/imagem/22-08-16_175326_M_logo-santander.svg); background-repeat: no-repeat; background-position-x: center; background-position-y: center; background-size: 90%;">
    </div>
    <div style ="background: #CC0100; width: 65%; height: 100px; display: inline-block; padding: 23px;">
      <h3 style ="color: white; text-align: center;">Nova Transação<h3>
    </div>
  </div>
  """

  def horarioInvalido(hora):
    hora = int(hora)
    
    if hora == 0:
      hora = 21
    elif hora == 1:
      hora = 22
    elif hora == 2:
      hora = 23
    elif hora == 3:
      hora = 0
    else:
      hora = hora - 3

    if hora <= 20 or hora < 6:
      return True
    else:
      return False

  def validarAudio():
    audio_source = '/content/audio.wav'
    data, sample_rate = librosa.load(audio_source, sr = None)
    sf.write(audio_source, data, sample_rate)
    audio = sr.AudioFile(audio_source)
    with audio as source:
      sr.Recognizer().adjust_for_ambient_noise(source, duration=0.2)
      texto = sr.Recognizer().record(source)

    retorno = False
    try:
      frase = sr.Recognizer().recognize_google(texto, language='pt-BR')
      if frase == 'não estou em casa':
        retorno = True
    except sr.UnknownValueError:
      retorno = 'erro'

    return retorno

    if sr.Recognizer().recognize_google(texto, language='pt-BR') == 'não estou em casa':
      return True
    else:
      return false

  # Cria display web
  st.markdown(html_temp, unsafe_allow_html = True) 

  
  col1, col2 = st.columns(2)
  with col1:
    chave = st.text_input('Chave PIX*')

  with col2:
    valor = st.text_input('Valor da transação*')

  st.text('Envie um audio com a frase "não estou em casa".')
  audio = st_audiorec()

  if isinstance(audio, dict):
    with st.spinner('Verificando audio!...'):
      ind, audio = zip(*audio['arr'].items())
      ind = np.array(ind, dtype=int) 
      audio = np.array(audio)          
      sorted_ints = audio[ind]
      stream = BytesIO(b"".join([int(v).to_bytes(1, "big") for v in sorted_ints]))
      wav_bytes = stream.read()

      with open('audio.wav', 'wb') as f:
        f.write(wav_bytes)

  def validation(chave, valor):
    if st.button("Transferir"):
      if valor.isnumeric() and int(valor) > 0:
        if horarioInvalido(datetime.datetime.now().strftime("%H")) and int(valor) > 3000:
          if validarAudio():
            st.success('Transação autorizada')  
          else:
            st.error('Transação não autorizada')
        else:
          st.success('Transação autorizada')
      else:
        st.warning('Preencha um valor válido')

  validation(chave, valor)

if __name__=='__main__': 
  main()

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-10-13 00:15:21.187 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 1.725s


your url is: https://crazy-vans-chew-35-204-5-81.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.204.5.81:8501

  Stopping...
^C
